<a href="https://colab.research.google.com/github/tiwariankit1001/Awesome-ML/blob/master/Tabular_Retrieval_Augmented_Generation_(Tabular_RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Retrieval-Augmented Generation (Tabular RAG)

Welcome back, and congratulations on mastering the foundational concepts of Tabular RAG from our first session! You've successfully built a "Cinephile Translator" that can converse with a single movie table, using an LLM to generate precise SQL queries. Now, as we begin Session 2, we’re ready to evolve our system into a truly powerful and production-ready data assistant. This session is dedicated to unlocking the full potential of our movie universe.

Our first major challenge is to move beyond isolated tables. The real magic lies in the relationships between our movie data—the connections between actors, directors, and genres. We will master the **SQL `JOIN`** operation, learning to seamlessly navigate these relationships to answer sophisticated multi-table queries. From there, we’ll dive into **advanced analytics**, using powerful clauses like `GROUP BY`, `HAVING`, and `ORDER BY` to extract deeper insights and rankings from our data, giving us the ability to go from simple facts to complex analytical summaries.

But we won’t stop at what pure SQL can do. We'll confront the **semantic gap**—the limitation of using keywords for conceptual searches—by introducing the concept of a **Hybrid RAG system**. We'll learn how to design an orchestration layer that intelligently combines structured SQL with a powerful vector-based semantic search, allowing us to answer highly subjective questions like finding a "thrilling sci-fi" film. Finally, we'll address the critical real-world challenges of **security, robustness, and performance**, ensuring our system is not only smart but also resilient, secure, and ready for deployment. This session will transform your foundational skills into the expertise needed to build a truly masterful Tabular RAG solution.

# Pre-req

The pre-req steps to use the Gemini API key with Python:

1.  **Get API Key:** Go to [aistudio.google.com](https://aistudio.google.com/), sign in, and create/copy your API key (secure it immediately).
2.  **Install Library:** In your terminal, run `pip install -q -U google-generativeai`.
3.  **Set API Key Securely:**
    * **Recommended:** Set it as an environment variable (e.g., `export GOOGLE_API_KEY="YOUR_KEY"` in terminal/shell config).
    * **Colab:** Use Colab Secrets to store `GOOGLE_API_KEY`.
4.  **Configure in Python:** In your script, retrieve the key (e.g., `os.getenv('GOOGLE_API_KEY')` or `userdata.get('GOOGLE_API_KEY')`) and then use `genai.configure(api_key=YOUR_KEY)`.

[Gemini API Key](https://aistudio.google.com/u/1/apikey)

In [ ]:
!pip install -qq -U pandas "numpy<2" matplotlib seaborn tabulate psycopg2-binary sqlalchemy randomname google-generativeai python-dotenv --force
!pip install --upgrade -qU langchain langchain-community sqlite-vec langchain-huggingface sentence-transformers
!pip install --upgrade numpy scipy pandas scikit-learn # and any other relevant data science libraries you use


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import os
from dotenv import load_dotenv  # For loading API key from a .env file
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import random
from datetime import datetime, timedelta
import randomname  # Make sure to install this: pip install random-name
from sqlalchemy import create_engine
from tqdm.auto import trange
load_dotenv()

model_name = 'gemini-2.5-flash' #@param

api_key = os.getenv("GOOGLE_API_KEY") or userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=api_key)

model = genai.GenerativeModel(model_name)

DB_NAME = 'movies.db' #@param

engine = create_engine(f'sqlite:///{DB_NAME}')

# api_key


In [ ]:
#@title Data Synthesis

# --- Configuration ---

NUM_DIRECTORS = 36 #@param{type:"slider", max:100, min:1, step:5}
NUM_ACTORS = 50 #@param{type:"slider", max:100, min:1, step:5}
NUM_STUDIOS = 20 #@param{type:"slider", max:100, min:1, step:5}
NUM_MOVIES = 100 #@param{type:"slider", max:1000, min:1, step:5}

# Total cast entries, not unique actor-movie pairs
NUM_CAST_MEMBERS = 716  #@param{type:"slider", max:1000, min:1, step:5}

# Total genre assignments
NUM_MOVIE_GENRES = 20  #@param{type:"slider", max:100, min:1, step:5}
NUM_REVIEWS = 7651 #@param{type:"slider", max:10000, min:1, step:5}

# Lists for random selection
NATIONALITIES = ["American", "British", "French", "Japanese",
                 "Indian", "Canadian", "German", "Korean", "Australian", "Spanish"]
GENRE_NAMES = ["Action", "Comedy", "Drama", "Sci-Fi", "Horror", "Thriller",
               "Animation", "Documentary", "Romance", "Fantasy", "Mystery", "Crime", "Adventure"]
ROLE_NAMES = ["Protagonist", "Sidekick", "Antagonist", "Supporting Role",
              "Cameo", "Hero", "Villain", "Love Interest", "Mentor"]
ADJECTIVES = randomname.ADJECTIVES

NOUNS = randomname.NOUNS

REVIEW_SNIPPETS = [
    "A must-watch! Captivating story and brilliant performances.",
    "Decent movie, but a bit slow in the middle.",
    "Absolutely terrible, wasted my time.",
    "Visually stunning, highly recommend!",
    "The plot twists kept me on the edge of my seat.",
    "An instant classic, I'll watch it again.",
    "Overrated and predictable.",
    "Great acting, weak script.",
    "Surprisingly good, exceeded my expectations.",
    "A true cinematic masterpiece."
]


# --- Helper Functions for Random Data Generation ---


def random_date(start_year, end_year):
    """Generates a random date string within a given year range."""
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    return (start_date + timedelta(days=random_number_of_days)).strftime('%Y-%m-%d')


def generate_random_title():
    """Generates a random movie title."""
    return f"{random.choice(ADJECTIVES)} {random.choice(NOUNS)} of the {random.choice(ADJECTIVES)} {random.choice(NOUNS)}".title().replace('_', ' ')


def generate_directors_data(num_directors):
    data = []
    for i in range(1, num_directors + 1):
        data.append({
            'director_id': i,
            'director_name': randomname.get_name(),
            'birth_date': random_date(1940, 2000),
            'nationality': random.choice(NATIONALITIES)
        })
    return pd.DataFrame(data)


def generate_actors_data(num_actors):
    data = []
    for i in range(1, num_actors + 1):
        data.append({
            'actor_id': i,
            'actor_name': randomname.get_name(),
            'birth_date': random_date(1950, 2005),
            'nationality': random.choice(NATIONALITIES)
        })
    return pd.DataFrame(data)


def generate_studios_data(num_studios):
    data = []
    for i in range(1, num_studios + 1):
        data.append({
            'studio_id': i,
            'studio_name': f"{random.choice(ADJECTIVES)} {random.choice(['Pictures', 'Studios', 'Films', 'Entertainment'])}",
            'founded_year': random.randint(1920, 2010),
            # Using nationalities list for countries too
            'country': random.choice(NATIONALITIES)
        })
    return pd.DataFrame(data)


def generate_genres_data(genre_names):
    data = [{'genre_id': i + 1, 'genre_name': name}
            for i, name in enumerate(genre_names)]
    return pd.DataFrame(data)


def generate_movies_data(num_movies, director_ids, studio_ids):
    data = []
    for i in range(1, num_movies + 1):
        budget = round(random.uniform(1_000_000, 200_000_000), 2)
        # Box office is at least budget
        box_office = round(budget * random.uniform(1.0, 10.0), 2)
        data.append({
            'movie_id': i,
            'title': generate_random_title(),
            'release_year': random.randint(1980, 2024),
            'duration_minutes': random.randint(80, 180),
            'synopsis': f"A thrilling story about {random.choice(ADJECTIVES).lower()} {random.choice(NOUNS).lower()} and its {random.choice(ROLE_NAMES).lower()}s. {random.choice(REVIEW_SNIPPETS)}",
            'budget_usd': budget,
            'box_office_gross_usd': box_office,
            'director_id': random.choice(director_ids),
            'studio_id': random.choice(studio_ids)
        })
    return pd.DataFrame(data)


def generate_cast_members_data(num_cast_members, movie_ids, actor_ids):
    data = []
    # To ensure unique movie-actor pairs for cast_members
    seen_pairs = set()
    cast_id_counter = 1

    while cast_id_counter <= num_cast_members:
        movie_id = random.choice(movie_ids)
        actor_id = random.choice(actor_ids)
        if (movie_id, actor_id) not in seen_pairs:
            seen_pairs.add((movie_id, actor_id))
            data.append({
                'cast_id': cast_id_counter,
                'movie_id': movie_id,
                'actor_id': actor_id,
                'role_name': random.choice(ROLE_NAMES)
            })
            cast_id_counter += 1
        # Prevent infinite loops if all combinations are exhausted
        if len(seen_pairs) == len(movie_ids) * len(actor_ids) and len(movie_ids) * len(actor_ids) < num_cast_members:
            print("Warning: Exhausted all unique movie-actor combinations before reaching desired cast_members count.")
            break
    return pd.DataFrame(data)


def generate_movie_genres_data(num_movie_genres, movie_ids, genre_ids):
    data = []
    # To ensure unique movie-genre pairs
    seen_pairs = set()

    while len(seen_pairs) < num_movie_genres:
        movie_id = random.choice(movie_ids)
        genre_id = random.choice(genre_ids)
        if (movie_id, genre_id) not in seen_pairs:
            seen_pairs.add((movie_id, genre_id))
            data.append({
                'movie_id': movie_id,
                'genre_id': genre_id
            })
        # Prevent infinite loops if all combinations are exhausted
        if len(seen_pairs) == len(movie_ids) * len(genre_ids) and len(movie_ids) * len(genre_ids) < num_movie_genres:
            print("Warning: Exhausted all unique movie-genre combinations before reaching desired movie_genres count.")
            break
    return pd.DataFrame(data)

def generate_reviews_data(num_reviews, movie_ids):
    data = []
    for i in trange(1, num_reviews + 1):
        data.append({
            'review_id': i,
            'movie_id': random.choice(movie_ids),
            'reviewer_name': randomname.get_name(),
            'rating': random.randint(1, 10),
            'review_text': random.choice(REVIEW_SNIPPETS) + " " + random.choice(REVIEW_SNIPPETS),
            'review_date': random_date(2000, 2024)
        })
    return pd.DataFrame(data)

# --- Main Data Generation and Database Population ---


def populate_movie_database(engine=engine):
    """Generates all data and populates the SQLite database."""
    print(f"Creating database: {DB_NAME}")

    # Generate data for each table
    print("Generating data...")
    df_directors = generate_directors_data(NUM_DIRECTORS)
    df_actors = generate_actors_data(NUM_ACTORS)
    df_studios = generate_studios_data(NUM_STUDIOS)
    df_genres = generate_genres_data(GENRE_NAMES)

    # Get IDs for foreign key references
    director_ids = df_directors['director_id'].tolist()
    actor_ids = df_actors['actor_id'].tolist()
    studio_ids = df_studios['studio_id'].tolist()
    genre_ids = df_genres['genre_id'].tolist()
    df_movies = generate_movies_data(NUM_MOVIES, director_ids, studio_ids)

    # Get movie IDs after generation
    movie_ids = df_movies['movie_id'].tolist()

    df_cast_members = generate_cast_members_data(
        NUM_CAST_MEMBERS, movie_ids, actor_ids)
    df_movie_genres = generate_movie_genres_data(
        NUM_MOVIE_GENRES, movie_ids, genre_ids)
    df_reviews = generate_reviews_data(NUM_REVIEWS, movie_ids)

    # Populate tables using pandas to_sql
    print("Populating tables...")

    df_directors.to_sql('directors', engine, if_exists='replace', index=False)
    df_actors.to_sql('actors', engine, if_exists='replace', index=False)
    df_studios.to_sql('studios', engine, if_exists='replace', index=False)
    df_genres.to_sql('genres', engine, if_exists='replace', index=False)
    df_movies.to_sql('movies', engine, if_exists='replace', index=False)
    df_cast_members.to_sql('cast_members', engine,
                           if_exists='replace', index=False)
    df_movie_genres.to_sql('movie_genres', engine,
                           if_exists='replace', index=False)
    df_reviews.to_sql('reviews', engine, if_exists='replace', index=False)

    print(
        f"Database '{DB_NAME}' populated successfully with randomized movie data!")
    print("\n--- Data Summary ---")
    print(f"Directors: {len(df_directors)} records")
    print(f"Actors: {len(df_actors)} records")
    print(f"Studios: {len(df_studios)} records")
    print(f"Genres: {len(df_genres)} records")
    print(f"Movies: {len(df_movies)} records")
    print(f"Cast Members: {len(df_cast_members)} records")
    print(f"Movie Genres: {len(df_movie_genres)} records")
    print(f"Reviews: {len(df_reviews)} records")


populate_movie_database(engine=engine)

Creating database: movies.db
Generating data...


  0%|          | 0/7651 [00:00<?, ?it/s]

Populating tables...
Database 'movies.db' populated successfully with randomized movie data!

--- Data Summary ---
Directors: 36 records
Actors: 50 records
Studios: 20 records
Genres: 13 records
Movies: 100 records
Cast Members: 716 records
Movie Genres: 20 records
Reviews: 7651 records


# Session 1

## Module 1: Foundational Concepts & Your First Query



### The Big Idea: The Data Divide & The Cinephile Translator
Welcome to the course! Our journey begins by understanding the central problem: the data divide. While our database holds all the intricate details about movies, an LLM alone can't access it. It's a classic case of having the knowledge but no way to read the book. We'll introduce the LLM as our Cinephile Translator—a powerful tool that can convert your natural language questions, like "What's the release year of 'Inception'?", into precise SQL queries that our SQLite database can understand.



### Setting Up Our Movie Archive
To get started, we need to set up our movie data archive. This section focuses on the practical steps of connecting to our SQLite database using Python's sqlite3 library. We'll confirm the database is open, understand the basic structure of our core tables (movies, actors, and directors), and confirm our data is ready to be queried. This foundational setup ensures we have a stable environment before we ask our first question.



### Detailed Movies Database Structure

This database schema is designed to represent various aspects of movies, their cast, crew, genres, studios, and user reviews. Each table serves a specific purpose, and relationships are established using primary and foreign keys.



#### 1\. `movies` Table

  * **Description:** Stores core information about each movie.
  * **Columns:**
      * `movie_id` (INTEGER, Primary Key): Unique identifier for the movie.
      * `title` (TEXT, NOT NULL): The title of the movie.
      * `release_year` (INTEGER): The year the movie was released.
      * `duration_minutes` (INTEGER): The length of the movie in minutes.
      * `synopsis` (TEXT): A brief summary of the movie's plot.
      * `budget_usd` (REAL): The production budget of the movie in USD.
      * `box_office_gross_usd` (REAL): The worldwide box office revenue in USD.
      * `director_id` (INTEGER, Foreign Key to `directors.director_id`): The director of the movie.
      * `studio_id` (INTEGER, Foreign Key to `studios.studio_id`): The studio that produced the movie.

#### 2\. `actors` Table

  * **Description:** Stores information about individual actors.
  * **Columns:**
      * `actor_id` (INTEGER, Primary Key): Unique identifier for the actor.
      * `actor_name` (TEXT, NOT NULL): The full name of the actor.
      * `birth_date` (TEXT): The actor's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The actor's nationality.

#### 3\. `directors` Table

  * **Description:** Stores information about movie directors.
  * **Columns:**
      * `director_id` (INTEGER, Primary Key): Unique identifier for the director.
      * `director_name` (TEXT, NOT NULL): The full name of the director.
      * `birth_date` (TEXT): The director's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The director's nationality.

#### 4\. `studios` Table

  * **Description:** Stores information about movie production studios.
  * **Columns:**
      * `studio_id` (INTEGER, Primary Key): Unique identifier for the studio.
      * `studio_name` (TEXT, NOT NULL): The name of the studio.
      * `founded_year` (INTEGER): The year the studio was founded.
      * `country` (TEXT): The country where the studio is primarily based.

#### 5\. `genres` Table

  * **Description:** Stores a list of movie genres.
  * **Columns:**
      * `genre_id` (INTEGER, Primary Key): Unique identifier for the genre.
      * `genre_name` (TEXT, NOT NULL, UNIQUE): The name of the genre (e.g., "Action", "Drama", "Sci-Fi").

#### 6\. `cast_members` Table (Junction Table)

  * **Description:** Links movies to actors, specifying the role an actor played in a particular movie. This handles the many-to-many relationship between `movies` and `actors`.
  * **Columns:**
      * `cast_id` (INTEGER, Primary Key): Unique identifier for a cast entry.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `actor_id` (INTEGER, Foreign Key to `actors.actor_id`): The actor ID.
      * `role_name` (TEXT): The name of the character played by the actor in that movie.

#### 7\. `movie_genres` Table (Junction Table)

  * **Description:** Links movies to genres, allowing a movie to have multiple genres. This handles the many-to-many relationship between `movies` and `genres`.
  * **Columns:**
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `genre_id` (INTEGER, Foreign Key to `genres.genre_id`): The genre ID.
      * (Composite Primary Key: `movie_id`, `genre_id` to ensure unique genre assignments per movie).

#### 8\. `reviews` Table

  * **Description:** Stores user reviews and ratings for movies.
  * **Columns:**
      * `review_id` (INTEGER, Primary Key): Unique identifier for the review.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie being reviewed.
      * `reviewer_name` (TEXT): The name or alias of the reviewer.
      * `rating` (INTEGER): The numerical rating given (e.g., 1-10).
      * `review_text` (TEXT): The full text of the review.
      * `review_date` (TEXT): The date the review was posted (YYYY-MM-DD format).



### ASCII-Based Database Connection Graph




This graph visually represents the tables and their relationships within the movie database schema. Primary Keys (PK) and Foreign Keys (FK) indicate how tables are linked.

```plaintext
+----------------+       +------------+                 +------------+      +------------+
|  directors.    |       |   actors   |                 |  studios   |      |   genres   |
+----------------+       +------------+                 +------------+      +------------+
| director_id PK |       | actor_id   | PK  <----+      | studio_id  | PK   | genre_id PK|
| director_name  |       | actor_name |          |      | studio_name|      | genre_name |
| birth_date     |       | birth_date |          |      | founded_yr |      +------------+
| nationality    |       | nationality|          |      | country    |          ^
+----------------+       +------------+          |      +------------+          |
      ^                     ^                    |       ^                      |
      |                     |                    |       |                      |
      |                     |                    |       |                      |
      |                     |                    |       |                      |
      | director_id FK.     | actor_id FK        |       | studio_id FK         | genre_id FK
      |                     |                    |       |                      |
+-------------------------------------------------------------------------+
|                                    movies                               |
+-------------------------------------------------------------------------+
| movie_id PK                                                             |
| title                                                                   |
| release_year                                                            |
| duration_minutes                                                        |
| synopsis                                                                |
| budget_usd                                                              |
| box_office_gross_usd                                                    |
| director_id FK -------------------------------------------------------->|
| studio_id FK ---------------------------------------------------------->|
+-------------------------------------------------------------------------+
      ^           ^                   ^
      |           |                   | movie_id FK
      |           |                   |
      |           |                   |
+-------------------+           +-------------------+           +-------------------+
|   cast_members    |           |    movie_genres   |           |      reviews      |
+-------------------+           +-------------------+           +-------------------+
| cast_id PK        |           | movie_id FK       |<--------+ | review_id PK      |
| movie_id FK       |<----------+ genre_id FK       |<--------+ | movie_id FK       |<--+
| actor_id FK       |           +-------------------+           | reviewer_name     |   |
| role_name         |                                           | rating            |   |
+-------------------+                                           | review_text       |   |
                                                                | review_date       |   |
                                                                +-------------------+   |
                                                                                        |
                                                                                        |
```

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import randomname  # Make sure to install this: pip install random-name
from sqlalchemy import create_engine

DB_NAME = 'movies.db'

engine = create_engine(f'sqlite:///{DB_NAME}')

## SQL Crash Course

### Before the LLM, You should know what is SQL

Learning SQL with SQLite is a fantastic way to grasp database fundamentals without the headache of setting up a complex server. SQLite is a self-contained, serverless database engine, which means it's incredibly easy to get started with. You can think of it as a database in a single file on your computer. 🚀

-----

## Why SQLite is Perfect for Beginners

  * **Zero Configuration:** There's no installation process or server to manage. You just download a single file and you're ready to go.
  * **Portability:** Since your entire database (tables, data, and all) is stored in one file, you can easily copy, move, or email it.
  * **Standard SQL:** SQLite uses a standard, widely understood version of the SQL language. The skills you learn with SQLite are directly transferable to more powerful database systems like PostgreSQL, MySQL, and SQL Server.

-----

## Getting Started

1.  **Download SQLite:** Go to the official [SQLite download page](https://www.sqlite.org/download.html) and grab the precompiled binary for your operating system. It's just a single executable file.
2.  **Open the Command Line:** Open your terminal or command prompt, navigate to the folder where you saved the SQLite file.
3.  **Create a Database:** To create and open a new database file, simply type:
    ```
    sqlite3 my_first_database.db
    ```
    This command starts the SQLite program and creates a file named `my_first_database.db`. You are now inside the SQLite shell, ready to enter SQL commands.

-----

## Core SQL Commands to Practice

Let's learn the basic SQL commands by creating and managing a simple `students` table.

### **CREATE TABLE**

This command builds the structure, or "schema," for your table. You define the columns and the type of data they will hold (e.g., `INTEGER`, `TEXT`, `REAL`).

```sql
CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    age INTEGER
);
```

Here, `id` is the **primary key**, which is a unique identifier for each row.

### **INSERT INTO**

Now that you have a table, you can add data (rows) into it using the `INSERT INTO` statement.

```sql
INSERT INTO students (first_name, last_name, age) VALUES ('Alex', 'Smith', 22);
INSERT INTO students (first_name, last_name, age) VALUES ('Jane', 'Doe', 21);
```

### **SELECT**

The `SELECT` command is used to retrieve data. It's the command you'll use most often.

  * **To see all data:** The asterisk `*` is a wildcard for "all columns."
    ```sql
    SELECT * FROM students;
    ```
  * **To filter data with `WHERE`:** Use the `WHERE` clause to specify conditions.
    ```sql
    SELECT first_name, age FROM students WHERE age > 21;
    ```
  * **To sort data with `ORDER BY`:**
    ```sql
    SELECT * FROM students ORDER BY last_name ASC; -- ASC for ascending, DESC for descending
    ```

### **UPDATE**

To modify data that's already in a table, use `UPDATE`. It's crucial to use a `WHERE` clause, otherwise, you'll update all rows in the table\!

```sql
UPDATE students
SET age = 23
WHERE first_name = 'Alex';
```

### **DELETE**

This command removes rows from a table. Just like `UPDATE`, always use a `WHERE` clause to specify which row(s) to delete.

```sql
DELETE FROM students WHERE id = 2;
```

### **JOIN**

Databases are powerful because they can relate different pieces of information. Let's create a `courses` table and link it to `students`.

```sql
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_name TEXT,
    FOREIGN KEY(student_id) REFERENCES students(id)
);

INSERT INTO courses (student_id, course_name) VALUES (1, 'Data Science');
```

Now, use a `JOIN` to retrieve the name of the student and the course they are enrolled in.

```sql
SELECT
    s.first_name,
    c.course_name
FROM
    students s
JOIN
    courses c ON s.id = c.student_id;
```

This query combines rows from `students` (aliased as `s`) and `courses` (aliased as `c`) where the student IDs match.

-----

## Next Steps

Once you're comfortable with these basic commands, you can explore more advanced topics:

  * **Aggregate Functions:** Learn functions like `COUNT()`, `SUM()`, `AVG()`, and `MAX()` to perform calculations on your data.
  * **GROUP BY:** Use this clause with aggregate functions to group rows that have the same values into summary rows.
  * **GUI Tools:** Try a graphical user interface like **DB Browser for SQLite**. It allows you to see your database structure, browse data, and test queries in a more visual environment, which can be very helpful for learning.

### Lets do this again in Python + SQLLite


You can master SQL concepts by using the powerful Python libraries **pandas** and **SQLAlchemy**. This approach lets you write familiar Python code to manipulate data and see how it translates directly to SQL database operations. It's a fantastic way to bridge the gap between data analysis in Python and structured database querying. 🐍

-----

## Why Use pandas and SQLAlchemy?

  * **pandas DataFrames:** A DataFrame is a two-dimensional table, much like a SQL table or a spreadsheet. Pandas provides intuitive, high-performance functions for filtering, transforming, and analyzing this data in memory.
  * **SQLAlchemy Core:** It provides a universal way to connect to almost any SQL database (we'll use SQLite). It lets you execute raw SQL strings or build queries programmatically.
  * **The Synergy:** You can create a DataFrame in pandas, push it to a SQL database with a single command, and then query it back into a DataFrame. This workflow makes it easy to understand how DataFrame operations correspond to SQL commands.

-----

## Getting Started

First, you need to install the libraries. You can also install `ipython-sql` to run SQL commands magically in environments like Jupyter notebooks.

```bash
pip install pandas sqlalchemy
```

Next, let's create a database "engine." We'll use an in-memory SQLite database, which is perfect for practice because it's temporary and requires no cleanup.

```python
import pandas as pd
from sqlalchemy import create_engine

# Create an in-memory SQLite database engine
engine = create_engine('sqlite:///:memory:')
```

This `engine` object is your connection to the database.

-----

## Mapping SQL to pandas Operations

Let's see how common SQL commands map to the pandas/SQLAlchemy workflow.

### **CREATE TABLE & INSERT INTO**

Instead of writing `CREATE` and `INSERT` manually, you can create a pandas DataFrame and use the `.to_sql()` method to do both at once.

```python
# Create a DataFrame
students_df = pd.DataFrame({
    'id': [1, 2],
    'first_name': ['Alex', 'Jane'],
    'last_name': ['Smith', 'Doe'],
    'age': [22, 21]
}).set_index('id') # Set a primary key

# Create a SQL table and insert data from the DataFrame
students_df.to_sql('students', con=engine, index_label='id')

print(engine.execute("SELECT * FROM students").fetchall())
# Output: [(1, 'Alex', 'Smith', 22), (2, 'Jane', 'Doe', 21)]
```

The `.to_sql()` method automatically generates and executes the `CREATE TABLE` and `INSERT INTO` statements based on the DataFrame's structure and data.

### **SELECT**

The `pd.read_sql()` function is the equivalent of `SELECT`. It runs a query and loads the result directly into a new DataFrame.

  * **To see all data (`SELECT *`):**

    ```python
    df_from_sql = pd.read_sql('students', con=engine, index_col='id')
    print(df_from_sql)
    ```

  * **To filter data (`WHERE`) and sort (`ORDER BY`):** You can write a full SQL query.

    ```python
    query = "SELECT first_name, age FROM students WHERE age > 21 ORDER BY first_name"
    filtered_df = pd.read_sql_query(query, con=engine)
    print(filtered_df)
    ```

### **UPDATE**

The "pandas way" to update is to modify the DataFrame first, then replace the entire SQL table with the updated DataFrame.

```python
# 1. Read the table into a DataFrame
update_df = pd.read_sql('students', con=engine, index_col='id')

# 2. Modify the DataFrame in pandas
update_df.loc[update_df['first_name'] == 'Alex', 'age'] = 23

# 3. Write the entire updated DataFrame back, replacing the old table
update_df.to_sql('students', con=engine, if_exists='replace', index_label='id')

print(engine.execute("SELECT * FROM students").fetchall())
# Output: [(1, 'Alex', 'Smith', 23), (2, 'Jane', 'Doe', 21)]
```

The `if_exists='replace'` argument drops the old table and creates a new one with the updated data.

### **DELETE**

Similarly, you can delete rows in pandas and then overwrite the database table.

```python
# 1. Read the table
delete_df = pd.read_sql('students', con=engine, index_col='id')

# 2. Drop the row in pandas (where id is 2)
delete_df = delete_df.drop(2)

# 3. Overwrite the table in the database
delete_df.to_sql('students', con=engine, if_exists='replace', index_label='id')

print(engine.execute("SELECT * FROM students").fetchall())
# Output: [(1, 'Alex', 'Smith', 23)]
```

### **JOIN**

You can perform joins using pandas' `merge` function or by writing a `JOIN` query directly.

```python
# Create a second DataFrame and push it to SQL
courses_df = pd.DataFrame({
    'course_id': [101, 102],
    'student_id': [1, 2],
    'course_name': ['Data Science', 'Web Development']
})
courses_df.to_sql('courses', con=engine, index=False)

# Now, perform the join with a SQL query
join_query = """
SELECT
    s.first_name,
    c.course_name
FROM
    students s
JOIN
    courses c ON s.id = c.student_id;
"""
joined_df = pd.read_sql_query(join_query, con=engine)
print(joined_df)
```

-----

## The Power of this Approach

Using pandas with SQLAlchemy provides a dynamic environment for learning SQL. You can seamlessly switch between high-level DataFrame manipulations and low-level SQL queries. Performing an operation in pandas and then figuring out the corresponding SQL query (or vice versa) is an incredibly effective way to solidify your understanding of both data manipulation paradigms.

## Exploring the data

In [ ]:

#@title Execute sample queries to confirm the database is populated and ready for use.

# count movies
movie_count = pd.read_sql_query("SELECT COUNT(*) AS count FROM movies", engine)
print(f"Total movies in database: {movie_count['count'][0]}")

# count directors
director_count = pd.read_sql_query(
    "SELECT COUNT(*) AS count FROM directors", engine)
print(f"Total directors in database: {director_count['count'][0]}")

# count actors
actor_count = pd.read_sql_query("SELECT COUNT(*) AS count FROM actors", engine)
print(f"Total actors in database: {actor_count['count'][0]}")

# count studios
studio_count = pd.read_sql_query(
    "SELECT COUNT(*) AS count FROM studios", engine)
print(f"Total studios in database: {studio_count['count'][0]}")

# count genres
genre_count = pd.read_sql_query("SELECT COUNT(*) AS count FROM genres", engine)
print(f"Total genres in database: {genre_count['count'][0]}")

Total movies in database: 100
Total directors in database: 36
Total actors in database: 50
Total studios in database: 20
Total genres in database: 13


In [ ]:

#@title Show Sample Movie Data

sample_movies = pd.read_sql_query("SELECT * FROM movies LIMIT 5", engine)
print("\nSample Movies Data:")
print(sample_movies.to_markdown(index=False))


Sample Movies Data:
|   movie_id | title                                               |   release_year |   duration_minutes | synopsis                                                                                                    |   budget_usd |   box_office_gross_usd |   director_id |   studio_id |
|-----------:|:----------------------------------------------------|---------------:|-------------------:|:------------------------------------------------------------------------------------------------------------|-------------:|-----------------------:|--------------:|------------:|
|          1 | Colors Screenwriting Of The Emotions Linear Algebra |           2011 |                142 | A thrilling story about geometry meat and its cameos. The plot twists kept me on the edge of my seat.       |  1.79738e+08 |            1.40486e+09 |            14 |           2 |
|          2 | Age Corporate Job Of The Speed Spirits              |           2003 |                157 | A thrilling

In [ ]:

#@title show sample directors data
sample_movies = pd.read_sql_query("SELECT * FROM directors LIMIT 5", engine)
print("\nSample Movies Data:")
print(sample_movies.to_markdown(index=False))


Sample Movies Data:
|   director_id | director_name    | birth_date   | nationality   |
|--------------:|:-----------------|:-------------|:--------------|
|             1 | generous-skylark | 1940-05-03   | Canadian      |
|             2 | optical-yard     | 1957-08-06   | Australian    |
|             3 | tall-plugin      | 1977-11-23   | Indian        |
|             4 | plumb-origin     | 1973-02-27   | Canadian      |
|             5 | candied-mountain | 1943-04-14   | Japanese      |


In [ ]:

#@title show sample directors data
sample_movies = pd.read_sql_query("SELECT * FROM reviews LIMIT 5", engine)
print("\nSample Reviews Data:")
print(sample_movies.to_markdown(index=False))


Sample Reviews Data:
|   review_id |   movie_id | reviewer_name      |   rating | review_text                                                                                                 | review_date   |
|------------:|-----------:|:-------------------|---------:|:------------------------------------------------------------------------------------------------------------|:--------------|
|           1 |         97 | investor-capital   |       10 | An instant classic, I'll watch it again. Visually stunning, highly recommend!                               | 2009-05-02    |
|           2 |         17 | oriented-occlusion |        1 | A must-watch! Captivating story and brilliant performances. The plot twists kept me on the edge of my seat. | 2014-12-20    |
|           3 |          4 | sizzling-citron    |        7 | Decent movie, but a bit slow in the middle. A must-watch! Captivating story and brilliant performances.     | 2007-04-18    |
|           4 |         64 | leather-karate

In [ ]:
#@title Sample Reviews per Movie Data
sample_movies = pd.read_sql_query("SELECT movie_id, count(1) as cnt FROM reviews group by movie_id order by 2 desc limit 20", engine)
print("\n:\n")
print(sample_movies.to_markdown(index=False))


:

|   movie_id |   cnt |
|-----------:|------:|
|         10 |   107 |
|         89 |    92 |
|         45 |    92 |
|         28 |    92 |
|         46 |    91 |
|          5 |    90 |
|         62 |    89 |
|         25 |    89 |
|         74 |    87 |
|         99 |    86 |
|         95 |    86 |
|         84 |    86 |
|         79 |    86 |
|         42 |    86 |
|         30 |    86 |
|         23 |    86 |
|         98 |    85 |
|         58 |    84 |
|         49 |    84 |
|         41 |    84 |


In [ ]:
#@title Sample Average Review per Movie Data
sample_movies = pd.read_sql_query("SELECT movie_id, avg(rating) as rating FROM reviews group by movie_id order by 2 desc limit 20", engine)
print(sample_movies.round(2).to_markdown(index=False))

|   movie_id |   rating |
|-----------:|---------:|
|         48 |     6.24 |
|         30 |     6.12 |
|         75 |     6.06 |
|         32 |     6.06 |
|         18 |     6    |
|          9 |     5.97 |
|         78 |     5.92 |
|         70 |     5.91 |
|         27 |     5.88 |
|         81 |     5.85 |
|         44 |     5.84 |
|          5 |     5.8  |
|         87 |     5.76 |
|         69 |     5.76 |
|         72 |     5.74 |
|         73 |     5.74 |
|         52 |     5.74 |
|         46 |     5.74 |
|         49 |     5.71 |
|         42 |     5.69 |


In [ ]:
#@title Sample Average Review per Movie Data:
sample_movies = pd.read_sql_query(
    """
SELECT
  movies.movie_id,
  movies.title,
  avg(rating) as rating
FROM reviews
inner join movies
on reviews.movie_id = movies.movie_id
group by movies.movie_id
order by 3 desc
limit 20
""", engine)
print(sample_movies.to_markdown(index=False))

|   movie_id | title                                                               |   rating |
|-----------:|:--------------------------------------------------------------------|---------:|
|         48 | Corporate Prefixes Radio Of The Algorithms Corporate Job            |  6.23611 |
|         30 | Algorithms Driving Of The Linguistics Travel                        |  6.11628 |
|         75 | Algorithms Geometry Of The Linguistics Birds                        |  6.06329 |
|         32 | Age Infrastructure Of The Weather Military Airforce                 |  6.06329 |
|         18 | Food Buildings Of The Materials Construction                        |  6       |
|          9 | Emotions Storage Of The Construction Music Production               |  5.96923 |
|         78 | Corporate Prefixes Buildings Of The Corporate Prefixes Architecture |  5.91566 |
|         70 | Quantity Water Of The Colors Gaming                                 |  5.90625 |
|         27 | Age Sports Of The Algorit

### Your First End-to-End Query
This is where we bring the core idea to life. We'll walk through a complete, end-to-end example of asking your first question. We'll introduce the basics of Prompt Engineering to guide the LLM, craft a simple request, see how the LLM generates the corresponding SQL, and then execute that query against our database. Finally, we'll synthesize the raw data into a clear, conversational answer for the user, demonstrating a complete Tabular RAG loop in a live demo.



In [ ]:
user_query =  'What was the budget of my studios movies in the year 2020?' #@param

# '''What are the top 5 movies in the database? Provide their titles, release years, and directors. Include a brief synopsis for each movie.'''

prompt = f"""
You are an AI assistant that can answer questions about a movie database. The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
"""

response = model.generate_content(prompt,).text

print(response)

```sql
SELECT
  SUM(M.budget)
FROM Movies AS M
JOIN Studios AS S
  ON M.studio_id = S.studio_id
WHERE
  S.studio_name = 'my studios' AND M.release_year = 2020;
```


In [ ]:
#@title ## Can We Execute This?
try:
  sql = response.split('```sql')[1].split('```')[0]
  print(pd.read_sql(sql, engine).to_markdown())
except Exception as e:
  print(e)
  print(response)

(sqlite3.OperationalError) no such column: M.budget
[SQL: 
SELECT
  SUM(M.budget)
FROM Movies AS M
JOIN Studios AS S
  ON M.studio_id = S.studio_id
WHERE
  S.studio_name = 'my studios' AND M.release_year = 2020;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
```sql
SELECT
  SUM(M.budget)
FROM Movies AS M
JOIN Studios AS S
  ON M.studio_id = S.studio_id
WHERE
  S.studio_name = 'my studios' AND M.release_year = 2020;
```


## Module 2: Unlocking Complexity with Schema & Aggregates



### The LLM's Script: The Importance of Database Schema
As a "cinephile translator," the LLM needs a script—the database schema—to truly understand our movie universe. We'll discuss why providing this schema context is crucial for generating accurate and sophisticated queries. We'll explore two primary methods for doing this: providing a static, inline schema and a more flexible, dynamic method that uses Python to discover the schema in real-time. This dynamic approach ensures our LLM always has the most up-to-date information, regardless of any changes to the database structure.



In [ ]:
user_query = '''What's the release year of 'X'?'''

prompt = f"""
User Query: {user_query}
"""

response = model.generate_content(prompt,).text

print(response)

To tell you the release year, I need to know what 'X' refers to!

Please tell me the name of the movie, song, book, game, product, or whatever 'X' is, and I'll be happy to find its release year for you.


#### Prompt Engineering 101: Guiding the LLM's Movie Knowledge

Prompt engineering is the art and science of crafting effective instructions for LLMs. To ensure our LLM generates accurate SQL for our movie database, we need to provide it with clear, concise guidance. This segment introduces the fundamental principles of prompt engineering, focusing on how to structure our requests to the LLM, provide necessary context, and specify the desired output format (SQL). It's about giving the LLM the right "script" to perform its translation task effectively for our movie data.

In [ ]:
user_query = '''What's the release year of 'X'?'''

prompt = f"""
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
"""

response = model.generate_content(prompt,).text

print(response)

```sql
SELECT release_year FROM Movies WHERE title = 'X';
```


#### Crafting the Request: Our Initial Simple Movie Prompt
Building on the principles of prompt engineering, this segment focuses on constructing our very first practical prompt. We'll design a simple, yet effective, instruction set for the LLM that includes our question ("What's the release year of 'Inception'?"). This initial prompt will be concise, aiming for a direct SQL output, and will serve as the foundation upon which we build more complex and sophisticated queries throughout the course. It's our first direct communication with the LLM to get movie data.

In [ ]:
movie_name = pd.read_sql('select title from movies', engine).sample(1).title.values[0]

print('Working with Movie Name: ', movie_name )

Working with Movie Name:  Emotions Software Of The Materials Algorithms


In [ ]:
# Sample Queries
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db
"""

user_queries = [
    f"What's the release year of '{movie_name}'?",
    # f"Who directed the movie '{movie_name}'?",
    # f"List all actors in the movie '{movie_name}'.",
    # f"What are the genres of the movie '{movie_name}'?",
]
sql_queries = []
for user_query in user_queries:
    resp = model.generate_content(prompt.format(user_query=user_query)).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    sql_queries.append(sql)
    print(user_query, ':', sql)
    print('-' * 80)
    print()

What's the release year of 'Emotions Software Of The Materials Algorithms'? : SELECT release_year FROM movies WHERE title = 'Emotions Software Of The Materials Algorithms';
--------------------------------------------------------------------------------



#### Executing the Query: Running LLM-Generated Movie SQL
Once our Large Language Model (LLM) has successfully translated your natural language question into a precise SQL query, the next critical step is to execute that query against our SQLite movie database. This segment focuses on the practical aspects of taking the LLM's generated SQL string and running it using Python's sqlite3 module. We'll ensure the query is syntactically correct and then retrieve the raw results directly from the database. This is where the theoretical understanding of SQL generation transforms into tangible data retrieval.

In [ ]:

for sql in sql_queries:
    try:
        print(pd.read_sql(sql, engine).to_markdown(index=False))
    except Exception as e:
        print(f"Error executing query: {e}")
    print('-' * 80)
    print()

|   release_year |
|---------------:|
|           1991 |
--------------------------------------------------------------------------------



In [ ]:
#@title #### Lets Retry with schema details/data dictionary

database_schema = '''

#### 1\. `movies` Table

  * **Description:** Stores core information about each movie.
  * **Columns:**
      * `movie_id` (INTEGER, Primary Key): Unique identifier for the movie.
      * `title` (TEXT, NOT NULL): The title of the movie.
      * `release_year` (INTEGER): The year the movie was released.
      * `duration_minutes` (INTEGER): The length of the movie in minutes.
      * `synopsis` (TEXT): A brief summary of the movie's plot.
      * `budget_usd` (REAL): The production budget of the movie in USD.
      * `box_office_gross_usd` (REAL): The worldwide box office revenue in USD.
      * `director_id` (INTEGER, Foreign Key to `directors.director_id`): The director of the movie.
      * `studio_id` (INTEGER, Foreign Key to `studios.studio_id`): The studio that produced the movie.

#### 2\. `actors` Table

  * **Description:** Stores information about individual actors.
  * **Columns:**
      * `actor_id` (INTEGER, Primary Key): Unique identifier for the actor.
      * `actor_name` (TEXT, NOT NULL): The full name of the actor.
      * `birth_date` (TEXT): The actor's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The actor's nationality.

#### 3\. `directors` Table

  * **Description:** Stores information about movie directors.
  * **Columns:**
      * `director_id` (INTEGER, Primary Key): Unique identifier for the director.
      * `director_name` (TEXT, NOT NULL): The full name of the director.
      * `birth_date` (TEXT): The director's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The director's nationality.

#### 4\. `studios` Table

  * **Description:** Stores information about movie production studios.
  * **Columns:**
      * `studio_id` (INTEGER, Primary Key): Unique identifier for the studio.
      * `studio_name` (TEXT, NOT NULL): The name of the studio.
      * `founded_year` (INTEGER): The year the studio was founded.
      * `country` (TEXT): The country where the studio is primarily based.

#### 5\. `genres` Table

  * **Description:** Stores a list of movie genres.
  * **Columns:**
      * `genre_id` (INTEGER, Primary Key): Unique identifier for the genre.
      * `genre_name` (TEXT, NOT NULL, UNIQUE): The name of the genre (e.g., "Action", "Drama", "Sci-Fi").

#### 6\. `cast_members` Table (Junction Table)

  * **Description:** Links movies to actors, specifying the role an actor played in a particular movie. This handles the many-to-many relationship between `movies` and `actors`.
  * **Columns:**
      * `cast_id` (INTEGER, Primary Key): Unique identifier for a cast entry.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `actor_id` (INTEGER, Foreign Key to `actors.actor_id`): The actor ID.
      * `role_name` (TEXT): The name of the character played by the actor in that movie.

#### 7\. `movie_genres` Table (Junction Table)

  * **Description:** Links movies to genres, allowing a movie to have multiple genres. This handles the many-to-many relationship between `movies` and `genres`.
  * **Columns:**
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `genre_id` (INTEGER, Foreign Key to `genres.genre_id`): The genre ID.
      * (Composite Primary Key: `movie_id`, `genre_id` to ensure unique genre assignments per movie).

#### 8\. `reviews` Table

  * **Description:** Stores user reviews and ratings for movies.
  * **Columns:**
      * `review_id` (INTEGER, Primary Key): Unique identifier for the review.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie being reviewed.
      * `reviewer_name` (TEXT): The name or alias of the reviewer.
      * `rating` (INTEGER): The numerical rating given (e.g., 1-10).
      * `review_text` (TEXT): The full text of the review.
      * `review_date` (TEXT): The date the review was posted (YYYY-MM-DD format).

'''

# Sample Queries
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db
schema details:
{database_schema}
"""

user_queries = [
    f"What's the release year of '{movie_name}'?",
    # f"Who directed the movie '{movie_name}'?",
    # f"List all actors in the movie '{movie_name}'.",
    # f"What are the genres of the movie '{movie_name}'?",
]
sql_queries = []
for user_query in user_queries:
    resp = model.generate_content(prompt.format(user_query=user_query, database_schema=database_schema)).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    sql_queries.append(sql)
    print(user_query, ':', sql)
    print('-' * 80)
    print()



What's the release year of 'Emotions Software Of The Materials Algorithms'? : SELECT
  release_year
FROM movies
WHERE
  title = 'Emotions Software Of The Materials Algorithms';
--------------------------------------------------------------------------------



In [ ]:
#@title retry execution

for sql in sql_queries:
    try:
        print(pd.read_sql(sql, engine).to_markdown(index=False))
    except Exception as e:
        print(f"Error executing query: {e}")
    print('-' * 80)
    print()

|   release_year |
|---------------:|
|           1991 |
--------------------------------------------------------------------------------



### Retrieving Specific Details & Summarizing Data
With the full database schema as our guide, we can tackle more complex questions. This section focuses on two key capabilities. First, we'll learn how to ask for multiple specific details at once, such as a movie's title and duration, and see how the LLM generates a multi-column SELECT statement. Second, we'll explore the power of SQL aggregate functions (COUNT, SUM, AVG, MIN, MAX) to summarize our data, answering broader questions like "How many movies are in the database?" or "What's the average movie duration?".



In [ ]:

user_queries = [
    f"what is the number of reviews for '{movie_name}'?",
    f"How many actors are there in '{movie_name}'?",
]
sql_queries = []

import time
for user_query in user_queries:
    resp = model.generate_content(prompt.format(user_query=user_query, database_schema=database_schema)).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    sql_queries.append(sql)
    print(user_query, ':', sql, '\n\n')
    try:
        print(pd.read_sql(sql, engine).to_markdown(index=False),'\n\n', )
    except Exception as e:
        print(f"Error executing query: {e}")
    print('-' * 80)
    print()
    time.sleep(2)

what is the number of reviews for 'Food Buildings Of The Materials Construction'? : SELECT
  COUNT(T1.review_id)
FROM reviews AS T1
INNER JOIN movies AS T2
  ON T1.movie_id = T2.movie_id
WHERE
  T2.title = 'Food Buildings Of The Materials Construction'; 


|   COUNT(T1.review_id) |
|----------------------:|
|                    76 | 


--------------------------------------------------------------------------------

How many actors are there in 'Food Buildings Of The Materials Construction'? : SELECT
  COUNT(T1.actor_id)
FROM cast_members AS T1
INNER JOIN movies AS T2
  ON T1.movie_id = T2.movie_id
WHERE
  T2.title = 'Food Buildings Of The Materials Construction'; 


|   COUNT(T1.actor_id) |
|---------------------:|
|                    3 | 


--------------------------------------------------------------------------------



### Live Demo: Advanced Queries & Aggregations on Demand
This live demonstration brings all these concepts together. We'll pose advanced natural language questions that require the LLM to generate more sophisticated SQL. You'll see the LLM generate multi-column queries, aggregate functions, and even filter data to retrieve the exact information we're looking for. This demo will visually reinforce how providing the schema empowers the LLM to provide richer and more insightful answers from our movie database.



In [ ]:
user_query = 'How many movies do we have in total per director' #@param

resp = model.generate_content(prompt.format(user_query=user_query, database_schema=database_schema)).text
sql = resp.strip().split('```sql')[1].split(
    '```')[0].strip()  # Extract SQL query from response
print(user_query, ':', sql, '\n\n')
try:
    print(pd.read_sql(sql, engine).to_markdown(index=False),'\n\n', )
except Exception as e:
    print(f"Error executing query: {e}")
print('-' * 80)
print()

How many movies do we have in total per director : SELECT
  d.director_name,
  COUNT(m.movie_id) AS total_movies
FROM directors AS d
JOIN movies AS m
  ON d.director_id = m.director_id
GROUP BY
  d.director_name
ORDER BY
  total_movies DESC; 


| director_name          |   total_movies |
|:-----------------------|---------------:|
| caramelized-tag        |              6 |
| rank-language          |              5 |
| loud-coffee            |              5 |
| brutal-suv             |              5 |
| sensitive-cyclotron    |              4 |
| parallel-width         |              4 |
| jet-path               |              4 |
| generous-skylark       |              4 |
| bone-pilaster          |              4 |
| best-hoops             |              4 |
| taxonomic-flamingo     |              3 |
| some-bulldozer         |              3 |
| partial-tributary      |              3 |
| oriented-instructor    |              3 |
| optical-yard           |              3 |
| noisy

In [ ]:
user_query = 'what are the top rated directors on the roster?' #@param
sql_break_query_prompt = '''
write a sql to detect the tables and views available to us.

database:

respond with only an executable query.
this outout needs to be used downstream in sql connection

'''
queries = model.generate_content(sql_break_query_prompt).text
print(queries)


```sql
SELECT
    TABLE_SCHEMA,
    TABLE_NAME,
    TABLE_TYPE
FROM
    INFORMATION_SCHEMA.TABLES
WHERE
    TABLE_TYPE IN ('BASE TABLE', 'VIEW');
```


In [ ]:

resp = model.generate_content(prompt.format(user_query=user_query, database_schema=database_schema)).text
sql = resp.strip().split('```sql')[1].split(
    '```')[0].strip()  # Extract SQL query from response
print(user_query, ':', sql, '\n\n')
top_directors = None

try:
    top_directors = pd.read_sql(sql, engine).to_markdown(index=False)
    print(pd.read_sql(sql, engine).to_markdown(index=False),'\n\n', )
except Exception as e:
    print(f"Error executing query: {e}")
print('-' * 80)
print()

What's the release year of 'Emotions Software Of The Materials Algorithms'? : SELECT
  release_year
FROM movies
WHERE
  title = 'Emotions Software Of The Materials Algorithms'; 


|   release_year |
|---------------:|
|           1991 | 


--------------------------------------------------------------------------------



In [ ]:
prompt = '''
You are an executive assistant answering to a CXO level boss.

summarize and respond in the best and most professional way possible.

query: {user_query}

context: {data_response}

additional context: {top_directors}
'''

data_response = pd.read_sql(sql, engine).to_markdown(index=False)

cxo_resp = model.generate_content(
    prompt.format(user_query=user_query, data_response=data_response, top_directors=top_directors)
).text

print(cxo_resp)

Certainly. Based on the provided data, here are the top-rated directors on the roster:

The highest-rated director is **candied-mountain** with an average rating of 6.0.

Following closely are:
*   **symmetric-fluid**: 5.798
*   **rank-language**: 5.755
*   **caramelized-tag**: 5.736
*   **oriented-instructor**: 5.655

I have rounded the ratings for conciseness. Please let me know if you would like to see the full list or require a deeper dive into any specific director's performance data.


## Module 3: Building a Robust & Resilient System



### Graceful Error Handling & Responding to the Unknown
Building a reliable system means preparing for things to go wrong. This section focuses on error handling and how to craft graceful responses. We'll cover scenarios where a query might return no results (and how to politely inform the user) and instances where the LLM might generate invalid SQL (and how to catch and handle these errors programmatically). A robust system anticipates these challenges to provide a smooth user experience.


In [ ]:

# Sample Queries
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db
schema details:
{database_schema}

"""

special_instructions = '''

SPECIAL INSTRUCTIONS:

MAKE AN ERROR IN THE SQL FORMATION, I WANT TO TEST ERROR HANDLING THE SQL QUERY
'''

user_queries = [
    f"What's the release year of '{movie_name}'?",
    # f"Who directed the movie '{movie_name}'?",
    # f"List all actors in the movie '{movie_name}'.",
    # f"What are the genres of the movie '{movie_name}'?",
]

for user_query in user_queries:
    resp = model.generate_content(
        prompt.format(user_query=user_query, database_schema=database_schema) + special_instructions
        ).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    print(user_query, ':', sql, '\n\n')
    try:
        print(pd.read_sql(sql, engine).to_markdown(index=False),'\n\n', )
    except Exception as e:
        print(f"Error executing query: {e}")
    print('-' * 80)
    print()


What's the release year of 'Food Buildings Of The Materials Construction'? : SELECT release_year
FROM moviess
WHERE title = 'Food Buildings Of The Materials Construction'; 


Error executing query: (sqlite3.OperationalError) no such table: moviess
[SQL: SELECT release_year
FROM moviess
WHERE title = 'Food Buildings Of The Materials Construction';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
--------------------------------------------------------------------------------



In [ ]:
error_handling_prompt = '''
You are a database response handler for business users.
Under stand the user query, the generated sql and the error that has occured.

provide a polite way to say that there was an error in the query and apologise.
your response may be seen by LLMs further down the line.

User Query: {user_query}

SQL Query: {sql}

Error: {error}
'''

for user_query in user_queries:
    resp = model.generate_content(
        prompt.format(user_query=user_query, database_schema=database_schema) + special_instructions
        ).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    print(user_query, ':', sql, '\n\n')
    db_response = None
    try:
        db_response = pd.read_sql(sql, engine).to_markdown(index=False)
    except Exception as e:
        db_response = model.generate_content(
            error_handling_prompt.format(
                error=str(e),
                user_query=user_query,
                sql=sql
              )
          ).text

    print(db_response,'\n\n', )
    print('-' * 80)
    print()


What's the release year of 'Food Buildings Of The Materials Construction'? : SELECT releas_year
FROM movies
WHERE title = 'Food Buildings Of The Materials Construction'; 


I apologize, but I encountered an error while trying to retrieve the release year for 'Food Buildings Of The Materials Construction'. It seems there was an issue with the specific column name I requested from the database.

I'm sorry for this inconvenience and am currently unable to provide the release year. Please let me know if you'd like me to try again, or if there's another way I can assist you. 


--------------------------------------------------------------------------------



In [ ]:
#@title Self Correction

error_handling_prompt = '''
You are a database response handler for business users.
Under stand the user query, the generated sql and the error that has occured.

if you can fix the query provide the fixed query and query alone. add in a comment: `--fixed query` at the end to mark fix

else provide a polite way to say that there was an error in the query and apologise.
your response may be seen by LLMs further down the line.

User Query: {user_query}

SQL Query: {sql}

schema: {database_schema}

Error: {error}
'''

for user_query in user_queries:
    resp = model.generate_content(
        prompt.format(user_query=user_query, database_schema=database_schema) + special_instructions
        ).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    print(user_query, ':', sql, '\n\n')
    db_response = None
    try:
        db_response = pd.read_sql(sql, engine).to_markdown(index=False)
    except Exception as e:
        db_response = model.generate_content(
            error_handling_prompt.format(
                error=str(e),
                user_query=user_query,
                sql=sql,
                database_schema=database_schema
              )
          ).text

    print(db_response,'\n\n', )
    print('-' * 80)
    print()


What's the release year of 'Food Buildings Of The Materials Construction'? : SELECT relese_year  -- Intentional misspelling of 'release_year'
FROM movies
WHERE title = 'Food Buildings Of The Materials Construction'; 


SELECT release_year
FROM movies
WHERE title = 'Food Buildings Of The Materials Construction'; --fixed query 


--------------------------------------------------------------------------------



In [ ]:
pd.read_sql(db_response, engine)

,release_year
0,1980



### Beyond Keywords: Bridging the Semantic Gap
Sometimes a simple keyword search isn't enough to capture the "movie magic" in a query. How do you find a film with an "intense atmosphere" or a "heartwarming plot"? This section introduces the concept of the semantic gap. We'll explore how movie embeddings—numerical representations of text—can bridge this gap, allowing us to move beyond exact matches and perform powerful semantic searches to find movies based on their conceptual meaning.

> Using [SQLLiteVec](https://python.langchain.com/docs/integrations/vectorstores/sqlitevec/)

In [ ]:
#@title Installing pre-reqs

!pip install --upgrade -qU langchain langchain-community sqlite-vec langchain-huggingface sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

#### Approach 1: Stuff everything to docs

In [ ]:
# r = model.generate_content(prompt.format(user_query='Build a single aggregation query to show all the data per movie in a single row', database_schema=database_schema)).text
# sql = r.split('```sql')[-1].split('```')[0]
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = 'movies.db' #@param

engine = create_engine(f'sqlite:///{DB_NAME}')

data = pd.read_sql(
    '''
SELECT
  m.movie_id,
  m.title,
  m.release_year,
  m.duration_minutes,
  m.synopsis,
  m.budget_usd,
  m.box_office_gross_usd,
  d.director_name,
  s.studio_name,
  GROUP_CONCAT( g.genre_name, ', ') AS genres,
  GROUP_CONCAT( a.actor_name || ' (as ' || cm.role_name || ')', '; ') AS cast_members,
  COUNT( r.review_id) AS total_reviews,
  ROUND(AVG(r.rating), 2) AS average_rating,
  GROUP_CONCAT( '(' || r.reviewer_name || ' - ' || r.rating || '/10): ' || r.review_text, '\n--- ') AS all_reviews_details
FROM movies AS m
LEFT JOIN directors AS d
  ON m.director_id = d.director_id
LEFT JOIN studios AS s
  ON m.studio_id = s.studio_id
LEFT JOIN movie_genres AS mg
  ON m.movie_id = mg.movie_id
LEFT JOIN genres AS g
  ON mg.genre_id = g.genre_id
LEFT JOIN cast_members AS cm
  ON m.movie_id = cm.movie_id
LEFT JOIN actors AS a
  ON cm.actor_id = a.actor_id
LEFT JOIN reviews AS r
  ON m.movie_id = r.movie_id
GROUP BY
  m.movie_id,
  m.title,
  m.release_year,
  m.duration_minutes,
  m.synopsis,
  m.budget_usd,
  m.box_office_gross_usd,
  d.director_name,
  s.studio_name
ORDER BY
  m.movie_id;
    ''',
    engine
)
data = [str(i) for i in data.to_dict('records')]

# len(data)

data[0]

'{\'movie_id\': 1, \'title\': \'Colors Screenwriting Of The Emotions Linear Algebra\', \'release_year\': 2011, \'duration_minutes\': 142, \'synopsis\': \'A thrilling story about geometry meat and its cameos. The plot twists kept me on the edge of my seat.\', \'budget_usd\': 179737961.19, \'box_office_gross_usd\': 1404855204.63, \'director_name\': \'rank-language\', \'studio_name\': \'weather Entertainment\', \'genres\': None, \'cast_members\': \'quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biography (as Villain); quiet-biog

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import SQLiteVec
from tqdm.auto import tqdm
from langchain_huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

connection = SQLiteVec.create_connection(db_file="vec.db")

vector_store = SQLiteVec(
    table="movie_intelligence",
    embedding=embedding_function,
    connection=connection,
    )


_ = vector_store.add_texts(texts=data)

In [ ]:
# query = "Funny Movie" #@param
# for doc in vector_store.similarity_search(query):
#   print(doc)

#### Approach 2: Distinct Vector DBs

In [ ]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import SQLiteVec
from tqdm.auto import tqdm
from langchain_huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

tables_to_vectorize = [
    'genres',
    # 'reviews',
    'movies',
]
stores = {}
for table_name in tqdm(tables_to_vectorize):
    data = pd.read_sql(
        f'''
        SELECT * FROM {table_name}
        ''',
        engine
    )

    print(data.head().to_markdown())

    data = [str(i) for i in data.to_dict('records')]

    connection = SQLiteVec.create_connection(db_file=f"{table_name}_vec.db")

    vector_store = SQLiteVec(
        table="intelligence",
        embedding=embedding_function,
        connection=connection,
        )

    _ = vector_store.add_texts(texts=data)
    stores[table_name] = vector_store

  0%|          | 0/2 [00:00<?, ?it/s]

|    |   genre_id | genre_name   |
|---:|-----------:|:-------------|
|  0 |          1 | Action       |
|  1 |          2 | Comedy       |
|  2 |          3 | Drama        |
|  3 |          4 | Sci-Fi       |
|  4 |          5 | Horror       |


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


|    |   movie_id | title                                              |   release_year |   duration_minutes | synopsis                                                                                                                 |   budget_usd |   box_office_gross_usd |   director_id |   studio_id |
|---:|-----------:|:---------------------------------------------------|---------------:|-------------------:|:-------------------------------------------------------------------------------------------------------------------------|-------------:|-----------------------:|--------------:|------------:|
|  0 |          1 | Music Theory Music Theory Of The Quantity Writing  |           1980 |                180 | A thrilling story about construction cats and its villains. An instant classic, I'll watch it again.                     |  1.17366e+08 |            1.03501e+09 |            10 |           1 |
|  1 |          2 | Corporate Prefixes Automobiles Of The Shape Design |           1983 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# vector_store = stores['reviews']
# query = 'Loved the movie'
# for doc in vector_store.similarity_search_with_score(query):
#   print(doc)

In [ ]:
vector_store = stores['genres']
query = 'Punching'

for doc in vector_store.similarity_search_with_score(query,):
  print(doc)



(Document(metadata={}, page_content="{'genre_id': 7, 'genre_name': 'Animation'}"), 1.3685766458511353)
(Document(metadata={}, page_content="{'genre_id': 7, 'genre_name': 'Animation'}"), 1.3685766458511353)
(Document(metadata={}, page_content="{'genre_id': 7, 'genre_name': 'Animation'}"), 1.3685766458511353)
(Document(metadata={}, page_content="{'genre_id': 7, 'genre_name': 'Animation'}"), 1.3685766458511353)


In [ ]:
vector_store = stores['movies']
query = 'Punching'

for doc in vector_store.similarity_search_with_score(query,):
  print(doc)



(Document(metadata={}, page_content='{\'movie_id\': 69, \'title\': \'Character Birds Of The Speed Cheese\', \'release_year\': 1997, \'duration_minutes\': 177, \'synopsis\': "A thrilling story about algorithms fish and its love interests. An instant classic, I\'ll watch it again.", \'budget_usd\': 75886332.01, \'box_office_gross_usd\': 158186375.83, \'director_id\': 31, \'studio_id\': 19}'), 1.3794889450073242)
(Document(metadata={}, page_content='{\'movie_id\': 69, \'title\': \'Character Birds Of The Speed Cheese\', \'release_year\': 1997, \'duration_minutes\': 177, \'synopsis\': "A thrilling story about algorithms fish and its love interests. An instant classic, I\'ll watch it again.", \'budget_usd\': 75886332.01, \'box_office_gross_usd\': 158186375.83, \'director_id\': 31, \'studio_id\': 19}'), 1.3794889450073242)
(Document(metadata={}, page_content="{'movie_id': 70, 'title': 'Appearance 3D Printing Of The Construction Fast Food', 'release_year': 2024, 'duration_minutes': 133, 'synop


## Module 4: Session Review & Next Steps

### Session 1 Recap & Key Takeaways
This section provides a quick recap of everything we've learned in the first session. We'll quickly review the journey from our initial concepts and first queries to unlocking the power of schema and aggregates. This summary will reinforce the foundational principles and key techniques, ensuring you're confident in our progress.

### Your First Steps & Preparing for Session 2
We'll wrap up the session by highlighting the key takeaways for your first steps in Tabular RAG. We'll then set the stage for Session 2, where we'll build on this foundation to tackle multi-table queries, advanced aggregations, and the conceptual semantic search we just discussed. Session 2 is where we'll truly unlock the full potential of our movie universe.

In [ ]:
#@markdown ---

# Session 2

## Module 1: Joining the Movie Universe (Multi-Table Queries)

### The Connected World: Speaking Across Movie Tables
So far, we've only talked to one movie table at a time. But the real power of a database comes from the relationships between its tables. Our movie universe is interconnected: a movie has a director, and a movie features several actors. This section introduces the concept of foreign keys as the "glue" that links our tables, allowing us to ask questions that span across the movies, actors, directors, cast_members, and movie_genres tables. We'll explore these relationships and set the stage for how we can navigate this web of data.

### The Glue: The Power of SQL JOINs
To navigate the relationships between our movie tables, we use the JOIN operation. This powerful SQL command allows us to combine rows from two or more tables based on a related column, like a movie_id or an actor_id. We'll focus on the two most common types: the INNER JOIN, which finds matching data in both tables (e.g., which actors are in a specific movie), and the LEFT JOIN, which returns all records from one table even if there are no matches in the other (e.g., all movies, even those without a genre assigned).

### Live Demo: Multi-Table Movie Query Magic
This live demonstration brings our JOIN concepts to life. We'll present a natural language challenge like, "List all actors who appeared in 'The Matrix'." You'll see how the LLM, now aware of our table relationships, generates a sophisticated SQL query that uses INNER JOIN to connect the movies table with the cast_members and actors tables. We'll execute this multi-table query and see how the LLM orchestrates the entire process to provide a comprehensive and accurate answer.




In [ ]:
movie_name = pd.read_sql('select title from movies', engine).sample(1).title.values[0]

print('Working with Movie Name: ', movie_name )

Working with Movie Name:  Colors 3D Graphics Of The Physics Minerals


In [ ]:
#@title Using DDL instead of data dictionary

ddl = '''
CREATE TABLE directors (
    director_id BIGINT,
    director_name TEXT,
    birth_date TEXT,
    nationality TEXT
)

CREATE TABLE movies (
    movie_id BIGINT,
    title TEXT,
    release_year BIGINT,
    duration_minutes BIGINT,
    synopsis TEXT,
    budget_usd FLOAT,
    box_office_gross_usd FLOAT,
    director_id BIGINT,
    studio_id BIGINT
)

CREATE TABLE studios (
    studio_id BIGINT,
    studio_name TEXT,
    founded_year BIGINT,
    country TEXT
)
'''

# Sample Queries

user_queries = [
    f"Which studios own the movie: '{movie_name}'?",
]


prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db

database schema:
{ddl}

USE ONLY THE SCHEMA PROVIDED. DO NOT ASSUME OTHER TABLE NAMES.

"""


rag_prompt = '''
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members,
and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

SQL Query: {resp}

Database Response:
{data_response}

'''

for user_query in user_queries:
    resp = model.generate_content(prompt.format(
        user_query=user_query, ddl=ddl)).text
    resp = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response

    try:

        data_response = pd.read_sql(resp, engine).to_markdown(index=False)

        llm_resp = model.generate_content(rag_prompt.format(
            user_query=user_query,
            resp=resp,
            data_response=data_response
        )
        ).text.strip()

        print(user_query, ':', llm_resp)

    except Exception as e:
        print(f"Error executing query: {e}")

    print('-' * 80)
    print()

Which studios own the movie: 'Food Meat Of The Materials Physics Waves'? : The movie 'Food Meat Of The Materials Physics Waves' is owned by **speed Pictures**.
--------------------------------------------------------------------------------



### Dynamic Schema Discovery (PRAGMA table_info for SQLite)

For more robust and scalable solutions, manually embedding schema in every prompt isn't practical. This segment introduces dynamic schema discovery, where our Python code programmatically extracts the database structure. Specifically for SQLite, we'll leverage the PRAGMA table_info command to query the database itself for its table and column metadata. This allows our system to automatically retrieve the latest schema information and pass it to the LLM, making our Tabular RAG application adaptable to database changes without manual prompt updates.

In [ ]:

def get_schema(table_name, engine=engine):
    """
    Retrieves the schema of a specified table in the SQLite database.

    Args:
        table_name (str): The name of the table to retrieve schema for.
        engine: SQLAlchemy engine connected to the SQLite database.

    Returns:
        str: Formatted string representing the table schema.
    """
    conn = engine.connect()
    columns_info = conn.exec_driver_sql(
        f'PRAGMA table_info("{table_name}")').fetchall()

    table_columns = [(col[1], col[2]) for col in columns_info]

    formatted_string = f"{table_name}:\n"
    for col_name, col_type in table_columns:
        formatted_string += f" {col_name} ({col_type})\n"

    return formatted_string


In [ ]:


ddl = f'''

{get_schema('movies', engine=engine)}

{get_schema('directors', engine=engine)}
'''

user_queries = [
    f"Which studios own the movie: '{movie_name}'?",
]


prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db

database schema:
{ddl}

"""


rag_prompt = '''
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members,
and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

SQL Query: {resp}

Database Response:
{data_response}

'''

for user_query in user_queries:
    resp = model.generate_content(prompt.format(
        user_query=user_query, ddl=ddl)).text
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response

    print('DEBUG:', sql)

    try:

        data_response = pd.read_sql(sql, engine).to_markdown(index=False)

        llm_resp = model.generate_content(rag_prompt.format(
            user_query=user_query,
            resp=sql,
            data_response=data_response
        )
        ).text.strip()

        print(user_query, ':', llm_resp)

    except Exception as e:
        print(f"Error executing query: {e}")
        print(sql)

    print('-' * 80)
    print()

DEBUG: SELECT
  T2.studio_name
FROM movies AS T1
INNER JOIN studios AS T2
  ON T1.studio_id = T2.studio_id
WHERE
  T1.title = 'Food Meat Of The Materials Physics Waves';
Which studios own the movie: 'Food Meat Of The Materials Physics Waves'? : The movie 'Food Meat Of The Materials Physics Waves' is owned by **speed Pictures**.
--------------------------------------------------------------------------------



## Fetching Schema with Python: Practical Example (Movie Tables)

Now that we understand the concept of dynamic schema discovery, this segment brings it to life with a practical Python example. We'll write and execute code that connects to our movies_db.db SQLite database and uses the PRAGMA table_info command to programmatically extract the names of all tables and their respective columns, along with their data types. This hands-on demonstration will show you exactly how to retrieve this vital "movie script" information directly from your database, preparing it for inclusion in our LLM prompts.


- Step 1: Identify tables
- Step 2: Check Table Schemas
- Step 3: Generate SQL
- Step 4: Retrieve data
- Step 5: RAG


In [ ]:
user_query = f"Which studios own the movie: '{movie_name}'?" #@param



In [ ]:
#@title Step 1
import json

tables = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", engine).to_markdown(index=False)

table_choice_prompt = f'''

You are an experienced Engineer and have been given the task of being the data retriever.

Given the user query, make a choice on the tables to be used to fetch the data and then be able to build the answer.

user query: {user_query}

table names: {tables}

Respond with the table names to be used as a json list of table names
'''

table_choice = model.generate_content(table_choice_prompt).text
json_parsed = json.loads(table_choice.split('```json')[-1].split('```')[0])

print(json_parsed)

['movies', 'studios']


In [ ]:
#@title Step 2

schemas = []

for table_name in json_parsed:
  schemas.append(get_schema(table_name, engine=engine))

schemas_prompt = '\n\n'.join(schemas)

print(schemas_prompt)

movies:
 movie_id (BIGINT)
 title (TEXT)
 release_year (BIGINT)
 duration_minutes (BIGINT)
 synopsis (TEXT)
 budget_usd (FLOAT)
 box_office_gross_usd (FLOAT)
 director_id (BIGINT)
 studio_id (BIGINT)


studios:
 studio_id (BIGINT)
 studio_name (TEXT)
 founded_year (BIGINT)
 country (TEXT)



In [ ]:
#@title Step 3, 4, 5
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db

database schema:
{ddl}

"""


rag_prompt = '''
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members,
and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

SQL Query: {resp}

Database Response:
{data_response}

'''


resp = model.generate_content(prompt.format(
    user_query=user_query, ddl=schemas_prompt)).text
sql = resp.strip().split('```sql')[1].split(
    '```')[0].strip()  # Extract SQL query from response

print('-'*80)
print('DEBUG:', sql)
print('-'*80)

try:

    data_response = pd.read_sql(sql, engine).to_markdown(index=False)

    llm_resp = model.generate_content(rag_prompt.format(
        user_query=user_query,
        resp=sql,
        data_response=data_response
    )
    ).text.strip()

    print(user_query, ':', llm_resp)

except Exception as e:
    print(f"Error executing query: {e}")
    print(sql)

print('-' * 80)
print()

--------------------------------------------------------------------------------
DEBUG: SELECT
  T2.studio_name
FROM movies AS T1
INNER JOIN studios AS T2
  ON T1.studio_id = T2.studio_id
WHERE
  T1.title = 'Food Meat Of The Materials Physics Waves';
--------------------------------------------------------------------------------
Which studios own the movie: 'Food Meat Of The Materials Physics Waves'? : The movie 'Food Meat Of The Materials Physics Waves' is owned by **speed Pictures**.
--------------------------------------------------------------------------------



## Module 2: Advanced Movie Analytics (Aggregations & Ranking)

### Beyond Simple Summaries: Advanced Aggregations
We've learned to get simple totals with COUNT, but what if we want more nuanced insights? This section moves beyond basic summary statistics to advanced aggregations. We'll introduce the GROUP BY clause, a fundamental tool that lets us partition our data into groups based on a specific column (e.g., group all movies by genre). We'll then use COUNT and other aggregates within these groups to answer questions like, "How many movies are in each genre?".

### Filtering & Ranking Movie Data
Once we've grouped our data, we often want to refine those results. This section introduces the HAVING clause, which allows us to filter the results of a GROUP BY operation based on a condition (e.g., "show me only genres with more than 10 movies"). We'll also cover the essential ORDER BY and LIMIT clauses, which let us sort our data and restrict the output to a specific number of records, enabling us to easily find top-ranked items like the "top 5 longest movies."

### Live Demo: Complex Aggregations & Ranking
This live demonstration combines everything we've learned about advanced analytics. We'll pose a series of challenging natural language questions that require the LLM to use GROUP BY, HAVING, ORDER BY, and LIMIT in a single query. We'll also explore temporal queries using SQLite's date functions to answer questions like "How many movies were released in each decade?". You'll see how the LLM orchestrates these complex operations to provide a powerful and precise analytical summary of our movie data.



In [ ]:
#@title Step 1
import json

def discover_tables(user_query, engine=engine):

  tables = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", engine).to_markdown(index=False)

  table_choice_prompt = f'''

  You are an experienced Engineer and have been given the task of being the data retriever.

  Given the user query, make a choice on the tables to be used to fetch the data and then be able to build the answer.

  user query: {user_query}

  table names: {tables}

  Respond with the table names to be used as a json list of table names
  '''

  table_choice = model.generate_content(table_choice_prompt).text
  json_parsed = json.loads(table_choice.split('```json')[-1].split('```')[0])

  return (json_parsed)

def build_ddl_prompt(json_parsed, engine=engine):
  schemas = []

  for table_name in json_parsed:
    schemas.append(get_schema(table_name, engine=engine))

  schemas_prompt = '\n\n'.join(schemas)

  return (schemas_prompt)


def r_of_rag(user_query, schemas_prompt, engine):

  prompt = """
  You are an AI assistant that can answer questions about a movie database.
  The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
  Your task is to answer the user's query based on the provided database information.

  User Query: {user_query}

  Generate a SQL query to retrieve the requested information from the database.
  database: sqlite:///movies.db

  database schema:
  {ddl}

  """

  resp = model.generate_content(prompt.format(
    user_query=user_query, ddl=schemas_prompt)).text
  try:

    sql = resp.strip().split('```sql')[1].split(
      '```')[0].strip()  # Extract SQL query from response

    print('-'*80)
    print(sql)
    print('-'*80)
    data_response = pd.read_sql(sql, engine).to_markdown(index=False)
    return data_response
  except Exception as e:
    print(e)
  return ""


def ag_of_rag(user_query, data_response, engine, chat_history):
  rag_prompt = '''
  You are an AI assistant that can answer questions about a movie database.
  The database contains information about movies, directors, actors, studios, genres, cast members,
  and reviews.
  Your task is to answer the user's query based on the provided database information.

  User Query: {user_query}

  Chat History:

  {chat_history}

  Database Response:
  {data_response}

  '''

  try:
      llm_resp = model.generate_content(rag_prompt.format(
          user_query=user_query,
          data_response=data_response,
          chat_history=chat_history
      )
      ).text.strip()

      return llm_resp

  except Exception as e:
      print(f"Error executing query: {e}")

user_queries = [
    f'I need to understand the popularity of the movie {movie_name}',
    f'What are the movies in the similar popularity scale?',
    f'What are the most common Genres for these movies?',
    f'Ok, I want to undertand what could be a common cast to such successful movies. Rank them by their performance in terms of overall movie rating'
]

history = []

for user_query in user_queries[:]:
  json_parsed = discover_tables(user_query)
  schemas_prompt = build_ddl_prompt(json_parsed)
  data_response = r_of_rag(user_query, schemas_prompt, engine)
  # print(data_response)
  history.append(f'data_context: \n{data_response}')
  resp = ag_of_rag(user_query, data_response, engine, history)
  history.append(f'user_query: {user_query}\nanswer: {resp}')
  print(user_query, ':', resp)
  print('-'*80)

--------------------------------------------------------------------------------
SELECT
  m.title,
  m.box_office_gross_usd,
  COUNT(r.review_id) AS number_of_reviews,
  AVG(r.rating) AS average_rating
FROM movies AS m
LEFT JOIN reviews AS r
  ON m.movie_id = r.movie_id
WHERE
  m.title = 'Colors 3D Graphics Of The Physics Minerals'
GROUP BY
  m.movie_id,
  m.title,
  m.box_office_gross_usd;
--------------------------------------------------------------------------------
I need to understand the popularity of the movie Colors 3D Graphics Of The Physics Minerals : Based on the information provided, here's an understanding of the popularity of "Colors 3D Graphics Of The Physics Minerals":

*   **Box Office:** The movie earned approximately **$116.82 million** at the box office, indicating a moderate level of commercial success.
*   **Reviews:** It received **81 reviews**, which is a relatively low number, suggesting it might not have been widely seen or discussed.
*   **Average Rating:** 

In [ ]:
[print(msg, '\n', '-'*80) for i, msg in enumerate(history) if i%2==1]

user_query: I need to understand the popularity of the movie Colors 3D Graphics Of The Physics Minerals
answer: Based on the information provided, here's an understanding of the popularity of "Colors 3D Graphics Of The Physics Minerals":

*   **Box Office:** The movie earned approximately **$116.82 million** at the box office, indicating a moderate level of commercial success.
*   **Reviews:** It received **81 reviews**, which is a relatively low number, suggesting it might not have been widely seen or discussed.
*   **Average Rating:** The movie holds an average rating of **5.72 out of 10**, indicating a somewhat mixed to slightly positive reception among those who reviewed it.

In summary, while it had a decent box office performance, the relatively low number of reviews and a middle-of-the-road rating suggest it was not a widely acclaimed or massively popular film. 
 --------------------------------------------------------------------------------
user_query: What are the movies in t

[None, None, None, None]

## Module 3: The Hybrid RAG System: Blending SQL & Semantic Search
### The Ultimate Challenge: Bridging the Semantic Gap
Pure SQL is fantastic for structured queries, but it struggles with subjective or conceptual questions like, "Find sci-fi movies with a thrilling plot." There's a semantic gap between a user's intent and what the database can understand through keywords alone. This section introduces the concept of a Hybrid Retrieval-Augmented Generation (RAG) system that blends structured SQL queries with a powerful vector-based semantic search to bridge this gap and provide truly intelligent answers.

### The Hybrid Workflow: Blending SQL & Vector Search
Our hybrid system will act as a smart orchestrator. When a user asks a complex question, the system will break it down. We'll demonstrate a scenario: "Find highly-rated sci-fi movies directed by Christopher Nolan with thrilling plots." The system will first use semantic search to find movie synopses with a "thrilling plot" and then use a structured SQL query to filter those results for "sci-fi genre" and "Christopher Nolan." This workflow shows how we can combine the best of both worlds to generate a comprehensive and accurate response.



In [ ]:
user_query = f'My kid likes Fighting Animated Movies. Recommend some which even i can watch. I love movies of actors in Horror and Comedy' #@param

In [ ]:
prompt = '''
You are my personal AI Assistant having access to a movie database.
You need to answer my queries and recommend me movies based
'''

p1 = '''
Given a user query, make a descision on whether you need to do Document Based RAG(`DRAG`) or Tabular RAG(`TRAG`).
based on the user query: `{user_query}` decide and respond back with only the keywords [`DRAG`, `TRAG`]. You can return both or just 1 as per requirement.

In case of `DRAG`, you have access to  `movies` and `genres` table indexes, make a cognitive choice and return the relevant indexes as well to use for my use case.
'''

resp = model.generate_content(p1.format(user_query=user_query)).text

print(resp)

context = ''
if 'DRAG' in resp:

  if 'reviews' in resp:
    vector_store = stores['reviews']
    context+='\n---\n'+'reviews'+'\n\n'+'\n'.join([doc.page_content  for doc in vector_store.similarity_search(user_query)])+'\n---\n'
  if 'movies' in resp:
    vector_store = stores['movies']
    context+='\n---\n'+'movies'+'\n\n'+'\n'.join([doc.page_content  for doc in vector_store.similarity_search(user_query)])+'\n---\n'
  if 'genres' in resp:
    vector_store = stores['genres']
    context+='\n---\n'+'genres'+'\n\n'+'\n'.join([doc.page_content  for doc in vector_store.similarity_search(user_query)])+'\n---\n'

if 'TRAG' in resp:
  # DO TRAG
  # adding DRAG Context to User Query
  uq = 'user_query: '+'\n\n'+user_query+'\n---\n'+context+'\n---\n'
  json_parsed = discover_tables(uq)
  schemas_prompt = build_ddl_prompt(json_parsed)
  data_response = r_of_rag(uq, schemas_prompt, engine)
  resp = ag_of_rag(uq, data_response, engine, [])
  print(user_query, ':', resp)

# print(context)

TRAG
--------------------------------------------------------------------------------
SELECT
  A.actor_name,
  AVG(MR.avg_rating) AS actor_overall_average_movie_rating
FROM actors AS A
JOIN cast_members AS CM
  ON A.actor_id = CM.actor_id
JOIN (
  SELECT
    M.movie_id,
    AVG(R.rating) AS avg_rating
  FROM movies AS M
  JOIN reviews AS R
    ON M.movie_id = R.movie_id
  GROUP BY
    M.movie_id
) AS MR
  ON CM.movie_id = MR.movie_id
GROUP BY
  A.actor_id,
  A.actor_name
ORDER BY
  actor_overall_average_movie_rating DESC;
--------------------------------------------------------------------------------
Ok, I want to undertand what could be a common cast to such successful movies. Rank them by their performance in terms of overall movie rating : Based on the overall average movie rating for the movies they have been in, here are the actors ranked from highest to lowest:

*   **zingy-archive**: 5.64
*   **bare-taping**: 5.63
*   **rigid-emmentaler**: 5.61
*   **objective-band**: 5.60
*   

## Module 4: Building Resilient & Scalable Movie RAG Systems

### Security, Robustness, & Performance
Building a production-ready system requires more than just clever queries. This section focuses on the critical aspects of security and performance. We'll address the ever-present threat of SQL injection, a vulnerability that can be exploited even in a simple SQLite database, and show how to prevent it using parameterized queries. We'll also discuss strategies for improving performance through indexing and how to handle ambiguous user queries gracefully, ensuring our system is not only smart but also safe, fast, and user-friendly.



In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Sample data
data = {
    'product_id': [101, 102, 201, 202, 301],
    'name': ['Laptop', 'Keyboard', 'Sci-Fi Novel', 'History Book', 'T-Shirt'],
    'category': ['Electronics', 'Electronics', 'Books', 'Books', 'Apparel']
}
df = pd.DataFrame(data)

# Create a SQLite database and table
engine = create_engine('sqlite:///products.db')
df.to_sql('products', engine, if_exists='replace', index=False)

print("Database 'products.db' created successfully.")

Database 'products.db' created successfully.


In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

engine = create_engine('sqlite:///products.db')

def get_products_vulnerable(llm_generated_filter: str):
    """
    VULNERABLE function that uses string formatting to build a query.
    This is dangerous!
    """
    # The developer trusts the LLM output and stitches it directly into the SQL.
    query = f"SELECT * FROM products WHERE {llm_generated_filter}"
    print(f"Executing Dangerous Query: {query}\n")

    with engine.connect() as conn:
        # Using pandas to read the result
        result_df = pd.read_sql(query, conn)
    return result_df

# --- The Attack ---

# Case 1: A normal, "good" prompt from the LLM
# User asks: "Find me all electronics" -> LLM generates: "category = 'Electronics'"
good_filter = "category = 'Electronics'"
print("--- Running with a normal, safe filter ---")
df_good = get_products_vulnerable(good_filter)
print(df_good)


# Case 2: A malicious prompt that tricks the LLM or is injected by a user
# User asks: "Find books, or whatever, just show me everything"
# A naive LLM or a direct attacker might generate the following payload:
malicious_filter = "category = 'Books' OR '1'='1'"

print("\n\n--- Running with a MALICIOUS filter (SQL Injection) ---")
df_hacked = get_products_vulnerable(malicious_filter)
print(df_hacked)

--- Running with a normal, safe filter ---
Executing Dangerous Query: SELECT * FROM products WHERE category = 'Electronics'

   product_id      name     category
0         101    Laptop  Electronics
1         102  Keyboard  Electronics


--- Running with a MALICIOUS filter (SQL Injection) ---
Executing Dangerous Query: SELECT * FROM products WHERE category = 'Books' OR '1'='1'

   product_id          name     category
0         101        Laptop  Electronics
1         102      Keyboard  Electronics
2         201  Sci-Fi Novel        Books
3         202  History Book        Books
4         301       T-Shirt      Apparel


In [ ]:
user_query = "I want category in 'Electronics' and '1' = '1'"


llm_sql =  model.generate_content(f'''
I am building a product sql with products table in mind.

My search is on exact level matches based on user query.
The User is the king/Queen and respect their wishes to the max.
Do not alter/modify or digress from user query at any cost

build a sql filter for the user query: {user_query}

pattern to follow: `SELECT * FROM products WHERE llm_generated_filter`

responsd with only the `llm_generated_filter`
''').text


llm_sql

"category IN ('Electronics') AND '1' = '1'"

In [ ]:
user_query = "I want category in 'Electronics' and '1' = '1'"

fix_prompt = '''
understand the query and rewrite against sql injection and other hacks.


'''

filter_str =  model.generate_content(fix_prompt).text


filter_str

## Module 5: Session 2 Recap & The Road Ahead

### Mastering Advanced Movie Tabular RAG
We've covered a lot in Session 2! This section will provide a final recap of our journey. We'll quickly review the journey from our initial concepts and first queries to unlocking the power of schema and aggregates, mastering advanced queries with JOINs and GROUP BY, and conceptualizing a hybrid RAG system. This summary will reinforce the foundational principles and key techniques, ensuring you're confident in our progress.

### Your Journey Continues
As we conclude the session, we'll offer some final thoughts on the future of Tabular RAG and how you can continue to apply these concepts to your own data. The skills you've developed are not limited to our movie database; they are the foundation for building intelligent systems that can converse with any structured data source. We’ll offer guidance on how to scale these concepts beyond SQLite and into larger, more complex database environments.